## importing 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,LSTM, Dense
from tensorflow.keras.models import load_model

2026-01-15 13:34:32.698975: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-15 13:34:32.886680: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-15 13:34:33.966250: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/satam/all_pro2/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: Future

In [2]:
df=pd.read_csv("qoute_dataset.csv")

## prepearing data

In [3]:
quote=df['quote']

In [4]:
df.shape

(3038, 2)

In [5]:
quote = quote.str.lower()

In [6]:
translator = str.maketrans('', '', string.punctuation)
quote = quote.str.translate(translator)

### Tokenizing

In [7]:
vocab_size=8978
Tokenizer=Tokenizer(num_words=vocab_size)
Tokenizer.fit_on_texts(quote)

In [8]:
print(len(Tokenizer.word_index))

8978


### save tokenizer

In [9]:
import pickle

# Save it properly
with open('Tokenizer.pkl', 'wb') as f:
    pickle.dump(Tokenizer, f)

# Verify it was saved (optional)
import os
print(f"File size: {os.path.getsize('Tokenizer.pkl')} bytes")

File size: 359295 bytes


In [10]:
sequences = Tokenizer.texts_to_sequences(quote)

In [11]:
X = []
y = []

for seq in sequences:
  for i in range(1,len(seq)):
    input_seq = seq[:i]
    output_seq = seq[i]
    X.append(input_seq)
    y.append(output_seq)

In [12]:
max_len = max(len(x) for x in X)
print(max_len)

745


In [13]:
X_padded = pad_sequences(X, maxlen=max_len, padding='pre')

In [14]:
y=np.array(y)

In [15]:
y_one_hot = to_categorical(y, num_classes=vocab_size)

In [16]:
y_one_hot[:1]

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 8978))

In [17]:
embedding_dim = 50
rnn_units = 128

## making model

In [18]:
rnn_model = Sequential()

In [19]:
embedding_dim = 50
rnn_units = 128

In [20]:
rnn_model.add(
    Embedding(input_dim=vocab_size,output_dim=embedding_dim, input_length=max_len)
)
rnn_model.add(
    SimpleRNN(units=rnn_units)
)
rnn_model.add(
    Dense(units=vocab_size,activation="softmax")
)

/home/satam/all_pro2/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [21]:

rnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
     

In [22]:
rnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
lstm_model = Sequential()
lstm_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
lstm_model.add(LSTM(units=rnn_units))
lstm_model.add(Dense(units=vocab_size, activation='softmax'))
     

lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [24]:
epochs=20
batch_size=128

## WARNING UNCOMMENT THIS IF YOU WANT  TO TRAIN MODEL IN YOUR MACHINE 

In [25]:
# lstm_model.fit(
#     X_padded,
#     y_one_hot,
#     epochs=epochs,
#     batch_size=batch_size,
# )

## predecting model

In [26]:
index_to_word={index: word for word, index in Tokenizer.word_index.items()}

In [27]:
lstm_model = load_model("/home/satam/Downloads/lstm_model.h5")

In [28]:
def predection(model,text,tokenizer,max_len):
    text=text.lower()
    seq1=tokenizer.texts_to_sequences([text])[0]
    seq=pad_sequences([seq1],maxlen=max_len,padding='pre')
    ##predictions
    # print(seq1)
    
    predict=model.predict(seq,verbose=0)
    predect_index=np.argmax(predict)
    # print(f'predict{predict}')
    return index_to_word[predect_index]

In [29]:
seed = "are you "
generate_text = predection(lstm_model,seed,Tokenizer,max_len)
print(generate_text)

implying


In [30]:
def generate_text(model,tokenizer,seed_text,max_len,n_words):
  for _ in range(n_words):
    next_word = predection(model,seed_text,tokenizer,max_len)
    if next_word == "":
      break
    seed_text += " " + next_word
  return seed_text

In [31]:
seed = "are you a "
generated_tex = generate_text(lstm_model,Tokenizer,seed,max_len,10)
print(generated_tex)

are you a  thousand times i wrote the less if it does not
